# Степени на свобода 

Манипулационният робот UR5 има 6 степени на свобода

In [2]:
links = 6 + 1;
joints = 6; 
sum_f_i = 6 * 1;
dof = 6 * (links - 1 - joints) + sum_f_i

dof = 6


# Помощни функции

get_skew_matrix(.) взема тримерен вектор v <br>
и връща косо-симетричната матрица, породена от него

In [3]:
function skew = get_skew_matrix(v)
    skew = [     0 -v(3)  v(2)
              v(3)     0 -v(1)
             -v(2)  v(1)     0 ];
end

get_vector_from_skew(.) взема косо-симетрична матрица M <br>
и връща вектора й

In [4]:
function v = get_vector_from_skew(M)
    v = [M(3, 2), M(1, 3), M(2, 1)];
end 

### Формула на Родригес 

get_exponent(. ,.) взема единичен тримерен вектор w (ос, около която ще е ротацията) и ъгъл Theta (ъгъл на ротиране) <br>
                   и връща ротационната матрица с експоненциални коодрдинати  

In [5]:
function R = get_exponent(w, Theta)
    skew_w = get_skew_matrix(w);
    skew_square = skew_w * skew_w;
    
    R = eye(3) + sin(Theta) * skew_w + (1 - cos(Theta)) * skew_square;
end

### Алгоритъм за пресмятане на логаритъм на ротационна матрица

get_axis_angle(.) взема ротационна матрица R <br>
                  и връща оста на ротация (единичен тримерен вектор) и ъгълът на ротация, който е в интервала [0, pi]

In [6]:
function [w, Theta] = get_axis_angle(R)
    if isdiag(R)
        w = NaN;
        Theta = 0; 
    elseif abs(trace(R) + 1) < 1e-4
        Theta = pi; 
        if R(3, 3) ~= -1 
            w = 1 / (2 * sqrt(1 + R(3, 3))) * [R(1, 3); R(2, 3); 1 + R(3, 3)];
        elseif R(2, 2) ~= -1
            w = 1 / (2 * sqrt(1 + R(2, 2))) * [R(1, 2); 1 + R(2, 2); R(3, 2)];
        else 
            w = 1 / (2 * sqrt(1 + R(1, 1))) * [1 + R(1, 1); R(2, 1); R(3, 1)];
        end
    else 
        Theta = acos((trace(R) - 1) / 2);
        w = get_vector_from_skew(1 / (2 * sin(Theta)) * (R - R'));
    end 
end

### Представяне на движение чрез експоненциялни координати

get_transformation_matrix(., .) взема винтово движение и изминато разстояние <br>
и връща матрицата на трансформацията

In [7]:
function T = get_transformation_matrix(twist, Theta)
    w = twist(1:3);
    v = twist(4:6);
    
    if abs(norm(w) - 1) < 1e-4
        skew_w = get_skew_matrix(w);
        skew_square = skew_w * skew_w;  
        
        R = get_exponent(w, Theta);
        p = ((eye(3) * Theta) ...
             + ((1 - cos(Theta)) * skew_w) ...
             + ((Theta - sin(Theta)) * skew_square)) ...
            * v;
    else 
        R = eye(3);
        p = v * Theta;
    end
    
    T = [  R   p 
         0 0 0 1 ];
end

get_rotation_matrix(.) взема трансформационна матрица <br>
и връща ротационната й матрица

In [8]:
function R = get_rotation_matrix(T)
    R = T(1:3, 1:3);
end

get_translation_vector(.) взема трансформационна матрица <br>
и връща вектора й на транслация

In [9]:
function p = get_translation_vector(T)
    p = T(1:3, 4);
end

# Алгоритъм за решаване на правата задача на кинематиката спрямо фиксирана пространствена отправна система {s}

solve_forward_S(., ., .) взема [S_i | S_i е винтовото движение в става i], нулевата матрица и списък от изминати разстояния <br>
                         и връща финалното положение на хващача 

Матрицата М описва положението на отправната система на хващача, {b}, спрямо отправната система в основата на робота, {s}. Вижда се, че посоката по у се е запазила, а х- и z-осите, съответно, са в противоположни посоки. Вижда се и векторът на транслация на {b} спрямо {s}; положението по z се запазва, по x транслацията е, колкото е радиусът на робота, а отместването по оста у беше съобразено по данни от симулатора и от DH параметрите.

In [10]:
# orientation of plane taken from the gripper coord system; translation was manually calculated 
M_sb = [ -1 0  0 0.817
          0 1  0 0.270
          0 0 -1 1.063
          0 0  0     1 ];
       
S_1 = [0, 0, 1,      0,      0,     0];  # (    0,      0,     0) x (0, 0, 1)
S_2 = [0, 1, 0, -1.163,      0,     0];  # (    0, 0.1317, 0.163) x (0, 1, 0)
S_3 = [0, 1, 0, -1.163,      0, 0.425];  # (0.425, 0.1317, 0.163) x (0, 1, 0)
S_4 = [0, 1, 0, -1.163,      0, 0.817];  # (0.817, 0.1317, 0.163) x (0, 1, 0)
S_5 = [0, 0, 1,  0.132, -0.817,     0];  # (0.817, 0.1317, 0.063) x (0, 0, 1)
S_6 = [0, 1, 0, -1.063,      0, 0.817];  # (0.817, 0.2307, 0.063) x (0, 1, 0)

S = [S_1' S_2' S_3' S_4' S_5' S_6'];

#Theta = [Theta_0, Theta_1, Theta_2, Theta_3, Theta_4, Theta_5];

function T = solve_forward_S(S, M, Theta)
    n = length(Theta);
    T = eye(4);
    
    for i = 1:n
        T = T * get_transformation_matrix(S(:, i), Theta(i));
    end
    
    T = T * M;
end

# Алгоритъм за решаване на правата задача на кинематиката спрямо отправна система {b}

solve_forward_B(., ., .) взема [В_i | В_i е винтовото движение в става i], нулевата матрица и списък от изминати разстояния <br>
                         и връща финалното положение на хващача 

In [11]:
#M_bs = [ -1 0  0  0.817
#         0 1  0 -0.270
#         0 0 -1  0.063
#         0 0  0      1 ];  

# B_0 = [0, 0, 1, -0.270, -0.817,     0];
# B_1 = [0, 1, 0,  0.100,      0, 0.817];
# B_2 = [0, 1, 0,  0.100,      0, 0.392];
# B_3 = [0, 1, 0,  0.100,      0,     0];
# B_4 = [0, 0, 1, -0.136,      0,     0];
# B_5 = [0, 1, 0,      0,      0,     0];

# B = [B_0' B_1' B_2' B_3' B_4' B_5'];

function T = solve_forward_B(B, M, Theta)
    n = length(Theta);
    T = M;
    
    for i = 1:n
        T = T * get_transformation_matrix(B(:, 1), Theta(i));
    end
    
end

# Алгоритъм за решаване на обратната задача на кинематиката

In [12]:
function Theta = solve_inverse(x, y, z, angle)
    Theta = zeros(6, 8);
    
    d_1 = 0.1625;
    d_5 = 0.0997;
    d_6 = 0.0996;
    d_4 = 0.1333;
    
    a_2 = -0.425;
    a_3 = -0.392;
    
    M_sb = [ -1 0  0 0.817
              0 1  0 0.270
              0 0 -1 1.063
              0 0  0     1 ];
    
    P_06 = [x; y; z];
    
    R = get_exponent([0, 0, 1], angle);
    
    # finding Theta(1)
    T_06 = [  R   P_06
            0 0 0   1 ]
    
    P_05 = T_06 * [0; 0; d_6; 1] - [0; 0; 0; 1]
    
    fi_1 = atan2(P_05(2), P_05(1));
    fi_2 = acos(d_4 / sqrt(P_05(1)^2 + P_05(2)^2)); 
    
    Theta(1, 1:4) = fi_1 + fi_2 + pi/2;
    Theta(1, 5:8) = fi_1 - fi_2 + pi/2;
    
    for i = 1:8
        Theta(1, i) = real(Theta(1, i));
        #if Theta(1, i) >= pi
        #    Theta(1, i) = Theta(1, i) - 2*pi;
        #end
    end
    
    # finding Theta(5)
    fn = [1, 5]; # indices 1:4 (incl) and indices 5:8 (incl) are for wrist flipped/not-flipped (up/down) accordingly 
    
    for i = 1:2
        coeff = fn(i);
        
        nominator = P_06(1) * sin(Theta(1, coeff)) - P_06(2) * cos(Theta(1, coeff)) - d_4;
        
        if (nominator < d_6 || abs(nominator - d_6 <= 1e-4)) # nominator / d_6 <= 1 
            Theta(5, coeff:coeff+1) = acos(nominator / d_6);
            Theta(5, coeff+2:coeff+3) = -acos(nominator / d_6);
        else 
            Theta(5, coeff:coeff+3) = 0;
        end
    end
    
    for i = 1:8
        Theta(5, i) = real(Theta(5, i));
        #if Theta(5, i) >= pi
        #    Theta(5, i) = Theta(5, i) - 2*pi;
        #end
    end
        
    # finding Theta(6)
    lr = [1, 3, 5, 7];
    
    for i = 1:4
        coeff = lr(i);
        
        sin_5 = sin(Theta(5, coeff));
        sin_1 = sin(Theta(1, coeff));
        cos_1 = cos(Theta(1, coeff));
        
        if abs(sin_5) > 1e-4
            Theta(6, coeff:coeff+1) = atan2((-T_06(2, 1) * sin_1 + T_06(2, 2) * cos_1) / sin_5, (T_06(1, 1) * sin_1 - T_06(1, 2) * cos_1) / sin_5);
        else 
            Theta(6, coeff:coeff+1) = 0;
        end
    end
    
    for i = 1:8
        Theta(6, i) = real(Theta(6, i));
        #if Theta(6, i) >= pi
        #    Theta(6, i) = Theta(6, i) - 2*pi;
        #end
    end
    
    # finding Theta(3)
    ud = [1, 3, 5, 7];
    
    for i = 1:4
        coeff = ud(i);
        
        T_01 = solve_forward_S([0; 0; 1;      0;      0;     0], M_sb, Theta(1, coeff));
        T_45 = solve_forward_S([0; 0; 1;  0.132; -0.817;     0], M_sb, Theta(5, coeff));
        T_56 = solve_forward_S([0; 1; 0; -1.063;      0; 0.817], M_sb, Theta(6, coeff));
        T_14 = T_01^-1 * T_06 * T_56^-1 * T_45^-1;
        
        P_14_xz_mod = sqrt(T_14(1, 4)^2 + T_14(3, 4)^2);
        
        nominator = P_14_xz_mod^2 - a_2^2 - a_3^3;
        arg = nominator / (2 * a_2 * a_3);
        
        if (abs(arg + 1) >= 1e-4 && abs(arg - 1) <= 1e-4)
            Theta(3, coeff) = acos(arg);
            Theta(3, coeff+1) = -acos(arg);
        else 
            Theta(3, coeff:coeff+1) = 0;
        end
    end
    
    for i = 1:8
        Theta(3, i) = real(Theta(3, i));
        #if Theta(3, i) >= pi
        #    Theta(3, i) = Theta(3, i) - 2*pi;
        #end
    end
        
    # finding Theta(2) and Theta(4)
    for i = 1:8
        T_01 = solve_forward_S([0; 0; 1;      0;      0;     0], M_sb, Theta(1, coeff));
        T_45 = solve_forward_S([0; 0; 1;  0.132; -0.817;     0], M_sb, Theta(5, coeff));
        T_56 = solve_forward_S([0; 1; 0; -1.063;      0; 0.817], M_sb, Theta(6, coeff));
        T_14 = T_01^-1 * T_06 * T_56^-1 * T_45^-1;
        
        P_14_xz_norm = sqrt(T_14(1, 4)^2 + T_14(3, 4)^2);
        
        psi_1 = atan2(-T_14(3, 4), -T_14(1, 4));
        psi_2 = asin((-a_3 * sin(Theta(3, i))) / P_14_xz_norm);
        
        #Theta(2)
        Theta(2, i) = psi_1 - psi_2;
        
        #Theta(4)
        T_12 = solve_forward_S([0; 1; 0; -1.163; 0;     0], M_sb, Theta(2, i));
        T_23 = solve_forward_S([0; 1; 0; -1.163; 0; 0.425], M_sb, Theta(3, i));
	    T_34 = T_23^-1 * T_12^-1 * T_14;
        
	    Theta(4, i) = atan2(T_34(2, 1), T_34(1, 1));
    end
    
    for i = 1:8
        Theta(2, i) = real(Theta(2, i));
        #if Theta(2, i) >= pi
        #    Theta(2, i) = Theta(2, i) - 2*pi;
        #end
        
        Theta(4, i) = real(Theta(4, i));
        #if Theta(4, i) >= pi
        #    Theta(4, i) = Theta(4, i) - 2*pi;
        #end
    end
end

In [13]:
ball =  [  0.4,  0.8, 1.04991];
crate = [-0.95, -0.5, 1];

In [14]:
function err = get_err(desire, result)
    x = result(1) - desire(1);
    y = result(2) - desire(2);
    z = result(3) - desire(3);
    err = sqrt(x^2 + y^2 + z^2);
end

In [16]:
Theta_inverse = solve_inverse(1.04991,  0.4, 0.8, 0)

for i = 1:8
    frwrd = solve_forward_S(S, M_sb, Theta_inverse(:, i));
    err = get_err([0.4, 0.8, 1.04991], frwrd(:, 1:3))
    if err <= 1е-1
        Theta_inverse(:, i)'
        frwrd
        [w_frwrd, theta_frwrd] = get_axis_angle(get_rotation_matrix(frwrd))
    end
end

T_06 =

   1.0000        0        0   1.0499
        0   1.0000        0   0.4000
        0        0   1.0000   0.8000
        0        0        0   1.0000

P_05 =

   1.0499
   0.4000
   0.8996
        0

Theta_inverse =

   3.3867   3.3867   3.3867   3.3867   0.4829   0.4829   0.4829   0.4829
  -2.1102  -2.1102  -2.1102  -2.1102  -2.1102  -2.1102  -2.1102  -2.1102
        0        0        0        0        0        0        0        0
   1.3077   1.3077   1.3077   1.3077   1.3077   1.3077   1.3077   1.3077
   1.5708   1.5708  -1.5708  -1.5708   1.5708   1.5708  -1.5708  -1.5708
  -1.8159  -1.8159   1.3257   1.3257   1.0879   1.0879  -2.0537  -2.0537

error: parse error:

  syntax error

>>>     if err <= 1е-1
                   ^
error: Theta_inverse(_,0+1i): subscripts must be real (forgot to initialize i or j?)
error: 'frwrd' undefined near line 1, column 1
error: 'frwrd' undefined near line 1, column 1
error: parse error:

  syntax error

>>>     end
          ^
error: parse erro